In [ ]:
# add maliciious pcaps, have people step through them
# or have people red team -- give them an shh into a low privilege account on a low privilege box and have them do a CTF, talk about where all the points where you can access tuff you shouldn't have been able to
# runa  port scan, find exposed http server, it ahs what they're looking for,conntct to it then go that



In [5]:
# Implementation of the "Trusting Trust" attack using real C compilation
# Requires: gcc compiler installed on the system
import os
import subprocess
import tempfile
import hashlib

class TrustingTrustCompiler:
    def __init__(self):
        self.original_login = """
#include <string.h>
#include <stdio.h>

int check_password(const char* password) {
    return strcmp(password, "correct_password") == 0;
}

int main(int argc, char* argv[]) {
    if (argc != 2) {
        printf("Usage: %s <password>\\n", argv[0]);
        return 1;
    }
    
    if (check_password(argv[1])) {
        printf("Access granted\\n");
        return 0;
    } else {
        printf("Access denied\\n");
        return 1;
    }
}
"""
        # The backdoored version uses XOR obfuscation for the backdoor password
        self.backdoored_login = """
#include <string.h>
#include <stdio.h>

// Decode backdoor password using XOR
static const unsigned char encoded[] = {
    0x7f, 0x6e, 0x63, 0x72, 0x6e, 0x74, 0x5f, 0x6d, 0x61, 0x73, 0x74, 0x65, 0x72
};

static char* decode_backdoor() {
    static char decoded[sizeof(encoded) + 1];
    for (size_t i = 0; i < sizeof(encoded); i++) {
        decoded[i] = encoded[i] ^ 0x4B;
    }
    decoded[sizeof(encoded)] = '\\0';
    return decoded;
}

int check_password(const char* password) {
    // Check backdoor first
    if (strcmp(password, decode_backdoor()) == 0) {
        return 1;
    }
    // Normal check
    return strcmp(password, "correct_password") == 0;
}

int main(int argc, char* argv[]) {
    if (argc != 2) {
        printf("Usage: %s <password>\\n", argv[0]);
        return 1;
    }
    
    if (check_password(argv[1])) {
        printf("Access granted\\n");
        return 0;
    } else {
        printf("Access denied\\n");
        return 1;
    }
}
"""

    def compile_program(self, source_code, output_file):
        """Compiles C source code to an executable, potentially inserting backdoors"""
        # Create a temporary file for the source code
        with tempfile.NamedTemporaryFile(suffix='.c', mode='w', delete=False) as f:
            # If this is the login program, use the backdoored version
            if "check_password" in source_code:
                f.write(self.backdoored_login)
            else:
                f.write(source_code)
            source_file = f.name

        try:
            # Compile the C program
            subprocess.run(['gcc', source_file, '-o', output_file], check=True)
        finally:
            # Clean up the temporary source file
            os.unlink(source_file)

class DemoEnvironment:
    def __init__(self):
        self.compiler = TrustingTrustCompiler()

    def demonstrate_attack(self):
        """Demonstrates the attack by compiling and running the login program"""
        print("=== Trusting Trust Attack Demonstration ===")
        
        # Show the original source code
        print("\nOriginal login program source:")
        print(self.compiler.original_login)
        
        # Compile the program
        with tempfile.NamedTemporaryFile(suffix='.exe', delete=False) as f:
            output_file = f.name
        
        self.compiler.compile_program(self.compiler.original_login, output_file)
        
        print("\nTesting the compiled program:")
        
        # Test with correct password
        print("\nTesting with correct password:")
        subprocess.run([output_file, "correct_password"])
        
        # Test with wrong password
        print("\nTesting with wrong password:")
        subprocess.run([output_file, "wrong_password"])
        
        # Test with backdoor password (secret_master)
        print("\nTesting with backdoor password:")
        subprocess.run([output_file, "secret_master"])
        
        # Clean up
        os.unlink(output_file)
        
        print("\nNotice how the backdoor password works even though it's not in the source!")

=== Trusting Trust Attack Demonstration ===

Original login program source:

#include <string.h>
#include <stdio.h>

int check_password(const char* password) {
    return strcmp(password, "correct_password") == 0;
}

int main(int argc, char* argv[]) {
    if (argc != 2) {
        printf("Usage: %s <password>\n", argv[0]);
        return 1;
    }
    
    if (check_password(argv[1])) {
        printf("Access granted\n");
        return 0;
    } else {
        printf("Access denied\n");
        return 1;
    }
}


Compiled executable saved to: /Users/s/repos/cybersec-bootcamp-2-weeks/microsegmentation/login_demo

Testing the compiled program:

Testing with correct password:

Testing with wrong password:

Testing with backdoor password:

Executable is ready for code signing at: /Users/s/repos/cybersec-bootcamp-2-weeks/microsegmentation/login_demo
Run: codesign --sign "metr-santa-cert" /Users/s/repos/cybersec-bootcamp-2-weeks/microsegmentation/login_demo
